# Vector search with LangChain (Azure AI Search)
이 노트북에서는 LangChain을 통해 Azure AI Search의 검색 기능들을 사용해 볼 것이다.

- 키워드 검색
- 벡터 검색
- 하이브리드 검색
- 의미 체계 하이브리드 검색



# 사전 준비
이 파이썬 예제를 실행하려면 다음과 같은 환경이 필요하다:
- Azure AI Search 리소스의 엔드포인트 및 쿼리 API 키
- Azure OpenAI Service를 사용할 수 있는 [승인 완료](https://aka.ms/oai/access)된 Azure 구독
- Azure OpenAI Service에 배포된 `text-embedding-ada-002` Embeddings 모델. 이 모델의 API 버전은 `2024-02-01`을 사용했다. 배포 이름은 모델 이름과 동일하게 `text-embedding-ada-002`로 명명했다.
- Azure OpenAI Service 연동 및 모델 정보
  - OpenAI API 키
  - OpenAI Embeddings 모델의 배포 이름
  - OpenAI API 버전
- Python (이 예제는 버전 3.12.4로 테스트 했다.)

이 예제에서는 Visual Studio Code와 [Jupyter extension](https://marketplace.visualstudio.com/items?itemName=ms-toolsai.jupyter)를 사용한다.

## 패키지 설치

In [ ]:
!pip install azure-search-documents==11.4.0


In [ ]:
!pip install azure-identity==1.17.1

In [ ]:
!pip install openai==1.35.3
!pip install langchain==0.2.5

In [ ]:
import azure.search.documents
print("azure.search.documents", azure.search.documents.__version__)
import openai
print("openai", openai.__version__)
import langchain
print("langchain", langchain.__version__)

## 라이브러리 및 환경변수 불러오기

In [ ]:
import os

os.environ["AZURESEARCH_FIELDS_CONTENT"]="content"
os.environ["AZURESEARCH_FIELDS_CONTENT_VECTOR"]="embedding"

from langchain_openai import AzureOpenAIEmbeddings
from langchain_community.vectorstores.azuresearch import AzureSearch

## 연동 설정

In [ ]:
# AI Search
service_endpoint: str = "<Your search service endpoint>"
service_admin_key: str = "<Your search service admin key>"

os.environ["AZURE_OPENAI_API_KEY"] = "Your OpenAI API Key"
os.environ["AZURE_OPENAI_ENDPOINT"] = "https://<Your OpenAI Service>.openai.azure.com/"

model: str = "text-embedding-ada-002"
index_name: str = "gptkbindex" # 00_DataIngest_AzureAISearch_PythonSDK.ipynb 에서 생성한 인덱스 이름

In [ ]:
embeddings = AzureOpenAIEmbeddings(
    azure_deployment=model,
    openai_api_version="2024-02-01"
)

vector_store: AzureSearch = AzureSearch(
    azure_search_endpoint=service_endpoint,
    azure_search_key=service_admin_key,
    index_name=index_name,
    embedding_function=embeddings.embed_query,
    semantic_configuration_name="default"
)

# 1. 키워드 검색
가장 간단한 검색인 키워드 검색부터 살펴보자. `ko.lucene`이라는 표준 한국어 분석기에 탑재된 사전 기반 토크나이저를 사용해서 토큰으로 분할한다. 분할된 토큰으로 역인덱스(역색인)를 생성한다.

그리고 TF/IDF 기반의 [BM25](https://en.wikipedia.org/wiki/Okapi_BM25) 채점 알고리즘을 사용해서 문장 내 토큰의 일치빈도를 보며 연관성 점수를 계산한다. 내부적으로는 토큰의 희소성과 문장의 밀도 등에도 가중치를 부여한다. 키워드 검색에서는 오타가 생기면 가령 아래의 예제처럼 **몽골**의 오타인 '몽돌'로 검색하면 기대한 결과가 나타나지 않거나 더 적게 나타날 수 있다.

In [ ]:
from langchain_community.retrievers import AzureAISearchRetriever

query = "몽돌 전쟁을 대비하기 위해 어떤 준비를 했나요?"
retriever = AzureAISearchRetriever(
    service_name="gptkb-xxxxxxxxx", #Your Azure AI Search service name
    index_name=index_name,
    api_key=service_admin_key,
    content_key="content",
    top_k=3,
)

docs = retriever.invoke(query)
for doc in docs:
    print(f"Source:{doc.metadata["sourcepage"]}")
    print(f"Score:{doc.metadata["@search.score"]}")
    print(f"Content: {doc.page_content}")
    

# 2. 벡터 유사도 검색

# 2.1. 간단한 벡터 검색
'몽**돌** 전쟁을 대비하기 위해 어떤 준비를 했나요?'라고 일부러 철자를 틀리게 입력한 쿼리로 검색을 해보자.
`text-embeddings-ada-002`로 생성한 벡터를 검색하면 일치하는 키워드를 찾는데 얽매이지 않고 문장의 유사도만을 고려해서 검색한다. 오타도 적절하게 무시하면서 검색하기 때문에 키워드 검색보다 더 나은 결과를 보여준다.

In [ ]:
query = "몽돌 전쟁을 대비하기 위해 어떤 준비를 했나요?"  

docs = vector_store.vector_search_with_score(
    query=query,
    k=3
)

for doc in docs:
    content = docs[0][0]
    score = docs[0][1]
    print(f"Source:{content.metadata["sourcepage"]}")
    print(f"Score:{score}")
    print(f"Content: {content.page_content}")

## 2.1.1. 다국어 처리
`text-embeddings-ada-002`의 다국어 처리 능력을 확인하기 위해 한국어 문서를 영어로 검색해보자.

In [ ]:
query = "What preparations were made to prepare for the Mongol invasion?"  

docs = vector_store.vector_search_with_score(
    query=query,
    k=3
)

for doc in docs:
    content = docs[0][0]
    score = docs[0][1]
    print(f"Source:{content.metadata["sourcepage"]}")
    print(f"Score:{score}")
    print(f"Content: {content.page_content}")

# 2.2. 하이브리드 검색
하이브리드 검색은 키워드 검색과 벡터 검색 모두를 쿼리에 사용한다. 키워드 검색 스코어를 구할 떄는 Okapi BM25 알고리즘을 사용해서 스코어를 계산하고, 벡터 검색에는 코사인 유사도를 기준으로 스코어를 계산한다. 이 서로 다른 계산 결과를 융합하는 방법으로는 [Reciprocal Rank Fusion(RRF)](https://learn.microsoft.com/azure/search/hybrid-search-ranking)을 사용한다. RRF는 두 방식으로 계산한 문서 랭크의 역수의 합을 구해서 문서 순위를 기준으로 어느 쪽의 랭크라도 상위에 있는 쪽이 스코어가 높아지는 구조다.

In [ ]:
query = "최씨정권을 타도하고 최고 권력자가 된 인물은?"  
# Perform a hybrid search
docs = vector_store.similarity_search(
    query=query,
    k=5,
    search_type="hybrid",
)

for doc in docs:
    print(f"Source:{doc.metadata["sourcepage"]}")
    print(f"Content: {doc.page_content}")

In [ ]:
query = "최씨정권을 타도하고 최고 권력자가 된 인물은?"  
# similarity_search에 search_type="hybrid"를 지정하는 것과 동일하다.
docs = vector_store.hybrid_search(
    query=query,
    k=5
)

for doc in docs:
    print(f"Source:{doc.metadata["sourcepage"]}")
    print(f"Content: {doc.page_content}")

# 2.3 의미 체계 하이브리드 검색
의미 체계 하이브리드 검색(하이브리드 검색 + 의미 체계 순위 지정)은 Azure AI Search에서만 지원하는 검색 기능으로, 하이브리드 검색과 검색 결과를 정확도가 높은 순서로 정렬하는 순위 재책정 기능(의미 체계 순위 지정)을 조합한 고도화된 검색 방법이다. 순위 재책정에는 마이크로소프트가 만든 언어 모델인 [Turing](https://techcommunity.microsoft.com/t5/ai-azure-ai-services-blog/introducing-multilingual-support-for-semantic-search-on-azure/ba-p/2385110) 모델을 사용한다.

In [ ]:
query = "무신정변을 일으켜 무신정권을 수립한 무신 3명은?"  

docs = vector_store.semantic_hybrid_search_with_score(
    query=query,
    k=5
)

for doc in docs:
    content = docs[0][0]
    score = docs[0][1]
    print(f"Source:{content.metadata["sourcepage"]}")
    print(f"Score:{score}")
    print(f"Content: {content.page_content}")